<a href="https://colab.research.google.com/github/LeeSeungwon89/Kaggle_Dacon_Practice/blob/main/2.%20Bike_Sharing_Demand_basline_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning이 발생하지 않도록 해줍니다.
!chmod 600 ~/.kaggle/kaggle.json

# 참가한 대회 리스트를 확인합니다.
!kaggle competitions list

In [ ]:
!kaggle competitions download -c bike-sharing-demand

  0% 0.00/189k [00:00<?, ?B/s]
100% 189k/189k [00:00<00:00, 71.2MB/s]


In [ ]:
!ls

bike-sharing-demand.zip  kaggle.json  sample_data


In [ ]:
!unzip bike-sharing-demand.zip

Archive:  bike-sharing-demand.zip
  inflating: sampleSubmission.csv    
  inflating: test.csv                
  inflating: train.csv               


# **1. 피처 엔지니어링**

In [ ]:
import numpy as np
import pandas as pd
import random

np.random.seed(2022)
random.seed(2022)

# 최대 행렬 수를 설정합니다.
pd.set_option('display.max_column', 50)
pd.set_option('display.max_rows', 50)

# 데이터를 읽습니다.
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sampleSubmission.csv')

## **1.1. 이상치 레코드 제거**

'weather' 피처에서 값이 4인 피처를 제거하겠습니다.

In [ ]:
train = train[train['weather']!=4]

## **1.2. 훈련 및 테스트 세트 결합**

피처 엔지니어링을 수행하기 위해 훈련 세트와 테스트 세트를 결합하겠습니다.

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,13.0,16.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,32.0,40.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,27.0,32.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,10.0,13.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17373,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17374,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,NaN,NaN
17375,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,NaN,NaN
17376,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,NaN,NaN


## **1.3. 피처 분할**

'datetime' 피처를 연, 월, 시, 요일 피처로 분할하겠습니다. 사용하기에 부적합한 날짜, 일, 분, 초 피처는 생성하지 않겠습니다.

In [ ]:
from datetime import datetime
import calendar

# 연, 월, 시 피처를 생성합니다.
all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])
all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])
all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])

# 날짜를 추출하고 날짜에 해당하는 요일을 숫자로 치환합니다.
all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])
all_data['day_of_week'] = all_data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').weekday())

# 날짜 피처를 삭제합니다.
all_data.drop(['datetime', 'date'], axis=1, inplace=True)

In [ ]:
all_data.head(1)

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,hour,day_of_week
0,1,0,0,1,9.84,14.395,81,0.0,3.0,13.0,16.0,2011,01,00,5


## **1.4. 불필요한 피처 제거**

불필요한 피처를 제거하겠습니다.

In [ ]:
feature_list = ['casual', 'windspeed', 'registered']
all_data.drop(feature_list, axis=1, inplace=True)

## **1.5. 피처 인코딩**

파생 피처인 'year', 'month', 'hour'는 명목형 피처입니다. 원-핫 인코딩을 적용하겠습니다.

In [ ]:
all_data_ohe = pd.get_dummies(all_data)
all_data_ohe.head()

,season,holiday,workingday,weather,temp,atemp,humidity,count,day_of_week,year_2011,year_2012,month_01,month_02,month_03,month_04,month_05,month_06,month_07,month_08,month_09,month_10,month_11,month_12,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23
0,1,0,0,1,9.84,14.395,81,16.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,9.02,13.635,80,40.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,1,9.02,13.635,80,32.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,1,9.84,14.395,75,13.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,9.84,14.395,75,1.0,5,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# **2. 모델링**

## **2.1. 데이터 준비**

데이터를 준비하겠습니다.

In [ ]:
train_num = len(train) # 훈련 세트 개수를 지정합니다.
X_train_df = all_data_ohe[:train_num].drop('count', axis=1) # 훈련 세트를 지정합니다.
X_test_df = all_data_ohe[train_num:].drop('count', axis=1) # 테스트 세트를 지정합니다.
y_train = train['count'] # 타깃값을 지정합니다.

타깃값에 로그 변환을 적용하겠습니다.

In [ ]:
log_y_train = np.log(y_train)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train_df, log_y_train,
                                                      test_size=0.2)

## **2.2. XGBoost**

기본 파라미터를 적용하겠습니다.

### **2.2.1. 모델 훈련**

모델을 훈련하겠습니다.

In [ ]:
from xgboost import XGBRegressor

xgb_reg_model = XGBRegressor()
xgb_reg_model.fit(X_train, y_train)

[05:28:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

### **2.2.2. 모델 성능 검증**

값을 예측하겠습니다.

In [ ]:
xgb_prediction_valid = xgb_reg_model.predict(X_valid)

이 문제의 측정 지표는 RMSLE입니다. RMSLE를 계산하는 함수를 선언하겠습니다.

In [ ]:
def rmsle(y, prediction, exponent=True): # 지수 변환을 기본값으로 지정합니다.
    # 타깃값에 지수 변환을 수행하길 원하면 지수 변환을 수행합니다.
    if exponent:
        y = np.exp(y)
        prediction = np.exp(prediction)

    # RMSLE 공식을 구현합니다.
    # 로그 변환을 수행하고 넘파이의 nan_to_num() 메서드를 사용하여 결측치를 0으로 변환합니다.
    log_y = np.nan_to_num(np.log(y + 1))
    log_prediction = np.nan_to_num(np.log(prediction + 1))
    result = np.sqrt(np.mean((log_y - log_prediction)**2))

    return result

RMSLE를 측정해 보겠습니다. 값이 작을수록 성능이 좋습니다.

In [ ]:
rmsle(y_valid, xgb_prediction_valid)

0.4592790874114472

테스트 세트를 예측하여 제출 파일을 생성하겠습니다.

In [ ]:
xgb_prediction_test = xgb_reg_model.predict(X_test_df)
submission['count'] = np.exp(xgb_prediction_test)
# submission.to_csv('Bike_Sharing_Demand_submission1.csv', index=False)

프라이빗 스코어는 0.51315입니다. 만족할 정도의 점수는 아닙니다.

## **2.3. LightGBM**

기본 파라미터를 적용하겠습니다. XGBoost 모델과 동일한 과정으로 수행합니다.

### **2.3.1. 모델 훈련**

In [ ]:
from lightgbm import LGBMRegressor

lgbm_reg_model = LGBMRegressor()
lgbm_reg_model.fit(X_train, y_train)

LGBMRegressor()

### **2.3.2. 모델 성능 검증**

In [ ]:
lgbm_prediction_valid = lgbm_reg_model.predict(X_valid)
rmsle(y_valid, lgbm_prediction_valid)

0.30632775865444384

일견 XGBoost보다 좋은 성능을 보입니다. 제출 파일을 생성하여 제출해 보겠습니다.

In [ ]:
lgbm_prediction_test = lgbm_reg_model.predict(X_test_df)
submission['count'] = np.exp(lgbm_prediction_test)
# submission.to_csv('Bike_Sharing_Demand_submission2.csv', index=False)

프라이빗 스코어는 0.40073입니다. 3,242팀 중 232위에 위치한 기록입니다. 

## **2.4. LinearRegression**

선형회귀 모델을 생성해 보겠습니다. 대개 앙상블 기반 결정트리 알고리즘이 더 좋은 성능을 발휘하므로 선형회귀 모델을 굳이 만들 필요는 없습니다. 

### **2.4.1. 모델 훈련**

In [ ]:
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()
linear_reg_model.fit(X_train, y_train)

LinearRegression()

### **2.4.2. 모델 성능 검증**

In [ ]:
linear_reg_prediction_valid = linear_reg_model.predict(X_valid)
rmsle(y_valid, linear_reg_prediction_valid)

0.5697838777919211

XGBoost보다도 낮은 성능을 보입니다.

# **3. 결론**

성능 개선 챕터에서 몇 가지 방안을 더 시도해보겠습니다. 

- 'month' 피처를 3개월씩 나누면 'season' 피처와 거의 비슷합니다. 원-핫 인코딩을 적용한 'month' 피처의 개수가 많으므로 'month' 피처를 제거하면 학습 속도 및 성능 제고를 도모해 볼 수 있습니다. 물론 큰 이점을 줄지는 미지수입니다. 그대로 사용하거나 제거할 경우의 차이를 살펴볼 필요가 있습니다. 이외 추가로 수행할 피처 엔지니어링은 없습니다.

- 모델링에서 시도한 세 알고리즘 중 앙상블 결정트리 알고리즘을 위주로 하이퍼 파라미터 튜닝을 시도하여 성능을 제고해 보겠습니다. 